In [ ]:
# 安装必要的依赖包
# e2b_code_interpreter: E2B代码解释器,用于在沙箱环境中执行代码
# e2b: E2B核心库,提供沙箱管理功能
# dotenv: 用于加载环境变量

! pip install e2b_code_interpreter e2b dotenv

In [ ]:
# 创建沙箱模版
# 使用E2B Template API定义自定义沙箱环境
# 设置环境变量和启动命令
from e2b import Template


In [ ]:
# 创建自定义沙箱并确定node、python、claude-cli、claude-agent-sdk版本
from e2b_code_interpreter import Sandbox
TEMPLATE_ID = '172m9tbyjat0ss16v9e8'
sbx = Sandbox.create(template=TEMPLATE_ID)
node_version = sbx.run_code("node -v", language="bash")
python_version = sbx.run_code("python3 --version", language="bash")
claude_cli_version = sbx.run_code("claude --version", language="bash")
claude_agent_sdk_version = sbx.run_code("python3 -c 'import claude_agent_sdk; print(claude_agent_sdk.__version__)'", language="bash")
print(node_version)
print(python_version)
print(claude_cli_version)
print(claude_agent_sdk_version)

In [ ]:
# 创建自定义沙箱并确定node、python、claude-cli、claude-agent-sdk版本
from e2b_code_interpreter import Sandbox
TEMPLATE_ID = '172m9tbyjat0ss16v9e8'
sbx = Sandbox.create(template=TEMPLATE_ID)
user = sbx.run_code("whoami", language="bash")
user_info = sbx.run_code("cat /etc/passwd | grep appuser", language="bash")
pwd = sbx.run_code("pwd", language="bash")
print(user)
print(user_info)
print(pwd)

In [ ]:
from e2b_code_interpreter import Sandbox
TEMPLATE_ID = '172m9tbyjat0ss16v9e8'
sbx = Sandbox.create(template=TEMPLATE_ID)
code = """
    from claude_agent_sdk import (
        AssistantMessage,
        ClaudeAgentOptions,
        ResultMessage,
        TextBlock,
        query,
    )
    print("=== Basic Example ===")

    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Write"],
        system_prompt={
            "type": "preset",
            "preset": "claude_code",
            "append": "在代码中始终包含详细的不带emoji表情的文档字符串和类型提示。"
        },
        max_turns=5,
        setting_sources=["project"],
        permission_mode="acceptEdits"
    )

    async for message in query(prompt="创建一个精美的html文件，介绍你自己，文件路径如下：/home/user/claude_code_introduction.html", options=options):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()
"""
result = sbx.run_code(code, language="python")
print(result)

content = sbx.files.read('/home/user/claude_code_introduction.html')
with open('claude_code_introduction.html', 'w') as file:
    file.write(content)

In [ ]:
from e2b_code_interpreter import Sandbox
TEMPLATE_ID = '172m9tbyjat0ss16v9e8'
sbx = Sandbox.create(template=TEMPLATE_ID, timeout=1800)
code="""
import anyio
from claude_agent_sdk import (
    AssistantMessage,
    ClaudeAgentOptions,
    ClaudeSDKClient,
    TextBlock,
)
options = ClaudeAgentOptions(
    allowed_tools=["Bash", "Edit", "Glob", "Grep", "Read", "Write"],
    system_prompt={
        "type": "preset",
        "preset": "claude_code",
        "append": \"""
## 简单应用开发规范

### 代码质量
- 代码简洁、可读
- 无需添加注释

### UI 要求
- 使用原生 CSS 创建现代化、美观的界面（可以使用 CSS Grid/Flexbox）
- 响应式设计
- 良好的用户体验

### 部署要求
- 开发完成后启动所有服务
- 明确说明服务运行的端口号
- 提供启动命令
\"""
    },
        permission_mode="bypassPermissions",
    )
async with ClaudeSDKClient(options) as client:
    
    # 具体的项目需求
    await client.query(\"""
创建一个简单的待办事项（Todo List）Web 应用。

### 技术栈
- 前端：原生 HTML + CSS + JavaScript（不使用任何框架）
- 后端：FastAPI + SQLite
- 无需用户认证系统

### 功能需求（保持简单）
1. 待办事项 CRUD
   - 添加待办事项（仅标题和完成状态）
   - 查看待办事项列表
   - 标记完成/未完成
   - 删除待办事项

2. UI 要求
   - 单页应用，美观的卡片式布局
   - 输入框 + 添加按钮
   - 待办事项列表（带复选框和删除按钮）
   - 使用原生 CSS 美化界面（可以使用 CSS Grid/Flexbox）
   - 简单的动画效果

3. 简化要求
   - 无需登录注册
   - 无需分类标签
   - 无需搜索功能
   - 无需分页（简单列表即可）

开发完成后启动前后端服务，前端运行在localhost的8001端口，后端运行在localhost的8002端口，直到验证无误。！！！使用中文回复！！！
\""")
        
    # 接收并打印响应
    async for message in client.receive_response():
        #print(message)
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(block.text)
"""
result = sbx.run_code(code, language="python")
print(result)
info = sbx.get_info()
print(info)
frontend_host = sbx.get_host(8001)
backend_host = sbx.get_host(8002)
url = f"https://{frontend_host}"
print(url)